## **Monte Carlo Methods**
The aim of this notebook is to implement different Monte Carlo methods and verify how these algorithms work over simple domains. Monte Carlo methods are good to deal with environments whose dynamics is not completely known.
These algorithm are able to learn effective policies from a direct experience with the environment (sampling the states and actions).

**Try these methods with different parameters!**

##Domain description
The domain used in this notebook is the classical **chessboard**, over which our aim is to calculate the best policy to move towards a specific tile (goal).

Possible **actions** and their decoding:
*  left  = 0
*  up    = 1
*  right = 2
*  down  = 3

In this notebook, the **policy** is represented as a **grid**, showing in each possible state the action to execute (or the action having higher probability to be executed in the probabilistic case).

The **reward** is -1 if the action is not feasible (the agent bumps on the border), +1 if the agent reaches the goal, 0 otherwise.

In the following lines it is proposed an implementation of the algorithms explained in the chapter 5 of "Reinforcement Learning: An Introduction" (2018).


Images links:
1. http://www.howtoreinforcementlearning.com/wp-content/uploads/2019/07/image-8.png
2. https://miro.medium.com/max/1490/1*nbe6oNqFQSIzB51Z8rDuag.pn
3. https://marcinbogdanski.github.io/rl-sketchpad/RL_An_Introduction_2018/assets/0504_OnPolicy_MC_Ctrl.png




In [0]:
#Import dependencies
import numpy as np

**First-visit MC prediction**


![alt text](http://www.howtoreinforcementlearning.com/wp-content/uploads/2019/07/image-8.png) 

In [0]:
#UTILITY FUNCTIONS FOR MC PREDICTION

def compute_episode(rows, cols, policy_grid, T, goal):
  """
  It returns an episode (S0,A0,R1,S1,A1,...,RT) computed applying the policy
  passed as parameter and starting from a random state s.

  :param rows: number of rows of the environment. 
  :param cols: number of cols of the environment. 
  :param policy_grid: grid showing the current greedy action to be executed.
  :param T: number of steps of the episode.
  :param goal: state (row,column) in which the agent receives the reward +1

  :returns: the sequences of states, actions and rewards of the episode.
  """
  S = []
  A = []
  R = []
  s = (np.random.randint(0,rows-1),np.random.randint(0,cols-1)) #initialize S_0

  for i in range(T):
    S.append(s)
    a = policy_grid[s[0],s[1]] #action of the policy to be executed

    if a == 0: #if left
      if s[1] == 0: #if the agent is on the left border
        r = -1 #the agent bump the border and s doesn't change
      else:
        s = (s[0], s[1]-1)
        r = 0

    elif a == 1: #if up
      if s[0] == 0: #if the agent is on the top border
        r = -1 #the agent bump the border and s doesn't change
      else:
        s = (s[0]-1, s[1])
        r = 0

    elif a == 2: #if right
      if s[1] == cols-1: #if the agent is on the right border
        r = -1 #the agent bump the border and s doesn't change
      else:
        s = (s[0], s[1]+1)
        r = 0

    elif a == 3: #if down
      if s[0] == rows-1: #if the agent is on the bottom border
        r = -1 #the agent bump the border and s doesn't change
      else:
        s = (s[0]+1, s[1])
        r = 0

    else:
      print("error in generate_actions!")

    if(s == goal):
      r = 1

    A.append(a)
    R.append(r)
    
  return S, A, R

def print_policy_grid(policy_grid):
  """
  Given a policy grid showing the action to be executed in each state, it prints
  the grid replacing the number of the action with an arrow representing the
  direction of the action selected by the policy.

  :param policy_grid: grid showing the current greedy action to be executed.
  """
  num_rows = len(policy_grid)
  num_cols = len(policy_grid[0])
  policy_grid_arrows = np.chararray((num_rows,num_cols), unicode = True)

  for r in range(num_rows):
    for c in range(num_cols):
      if policy_grid[r,c] == 0: #LEFT action
        policy_grid_arrows[r,c] = u'\u2190'
      elif policy_grid[r,c] == 1: #UP action
        policy_grid_arrows[r,c] = u'\u2191'
      elif policy_grid[r,c] == 2: #RIGHT action
        policy_grid_arrows[r,c] = u'\u2192'
      elif policy_grid[r,c] == 3: #DOWN action
        policy_grid_arrows[r,c] = u'\u2193'
      else:
        policy_grid_arrows[r,c] = "e" #error
  
  print("Policy grid:")
  print(policy_grid_arrows)

In [0]:
def first_visit_MC_prediction(policy = None, grid_rows = 6, grid_cols = 6, n_episodes = 1000, T = 10, gamma = 0.9, goal = (0,0)):
  """
  It executes the algorithm of first-visit Monte Carlo prediction for a policy.

  :param policy: policy to be evaluated (represented by a grid (grid_rows, grid_cols)).
  :param rows: number of rows of the environment. 
  :param cols: number of cols of the environment. 
  :param n_episodes: number of episodes the algorithm will compute.
  :param T: number of steps of each episode.
  :param gamma: discount parameter.
  :param goal: state (row,column) in which the agent receives the reward +1.
  """
  #Initializations:

  #Create policy to be evaluated (-1 if action is not feasible, 1 if the action brings to the goal, 0 otherwise)
  policy_grid = policy
  if(policy is None):
    print("No policy passed as param... Generating random policy grid...")
    policy_grid = np.random.randint(0,3,size=(grid_rows,grid_cols))
  print_policy_grid(policy_grid)

  #initialize arbitrarily V(s)
  V_s = np.random.uniform(0,0,(grid_rows,grid_cols))
  print("\nV(s):")
  print(V_s)

  #Create the empty list Returns(s)
  returns_s = {}
  for r in range(grid_rows):
    for c in range(grid_cols):
      returns_s['s_'+str(r)+'_'+str(c)] = []

  #Loop for each episode
  for i in range(n_episodes):
    G = 0
    S, A, R = compute_episode(grid_rows, grid_cols, policy_grid, T, goal) #generate episode

    for t in range(T-1,0,-1): #for each step of the episode
      s = S[t]
      G = gamma*G + R[t]
      if s not in S[t-1:-1:-1]: #removing this check, first-visit becomes every-visit!
        returns_s['s_'+str(s[0])+'_'+str(s[1])].append(G)
        V_s[s[0],s[1]] = np.mean(returns_s['s_'+str(s[0])+'_'+str(s[1])])

  print("\nV(s) at the episode", n_episodes,"goal in (",goal[0],",",goal[1],")\n")
  print(V_s)


In [20]:
#TEST

#create a random policy
grid_rows,grid_cols = 6,6
policy_grid = np.random.randint(0,3,size=(grid_rows,grid_cols))

#execution of MC prediction
first_visit_MC_prediction(grid_rows=5, grid_cols=5, n_episodes=5000, T=15)

No policy passed as param... Generating random policy grid...
Policy grid:
[['↑' '→' '↑' '←' '←']
 ['↑' '→' '→' '↑' '↑']
 ['→' '←' '→' '→' '→']
 ['↑' '→' '↑' '→' '←']
 ['←' '→' '↑' '←' '←']]

V(s):
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]

V(s) at the episode 5000 goal in ( 0 , 0 )

[[ 5.04207951  0.         -4.85474807 -6.45682972  0.        ]
 [ 0.          0.         -5.00232075 -5.68899009  0.        ]
 [ 0.          0.         -5.69282773 -6.45828817 -4.75291667]
 [ 0.          0.         -5.00232075  0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]]


**Monte Carlo ES (Exploring Starts)**


![alt text](https://miro.medium.com/max/1490/1*nbe6oNqFQSIzB51Z8rDuag.png)

In [0]:
#UTILITY FUNCTIONS FOR MC EXPLORING STARTS

#compute episode with exploring starts assumption:
#all pairs (s,a) have p > 0 to be S_0 and A_0
def compute_episode_ES(rows, cols, policy_grid, T, goal):
  """
  It returns an episode (S0,A0,R1,S1,A1,...,RT) computed applying the policy
  passed as parameter and the exploring starts assumption.

  :param rows: number of rows of the environment. 
  :param cols: number of cols of the environment. 
  :param policy_grid: grid showing the current greedy action to be executed.
  :param T: number of steps of the episode.
  :param goal: state (row,column) in which the agent receives the reward +1

  :returns: the sequences of states, actions and rewards of the episode.
  """
  S = []
  A = []
  R = []
  s = (np.random.randint(0,rows-1),np.random.randint(0,cols-1)) #initialize S_0

  for i in range(T):
    S.append(s)

    if i == 0:
      a = np.random.randint(0,3) #exploring starts assumption
    else:
      a = policy_grid[s[0],s[1]] #action of the policy to be executed

    if a == 0: #if left
      if s[1] == 0: #if the agent is on the left border
        r = -1 #the agent bump the border and s doesn't change
      else:
        s = (s[0], s[1]-1)
        r = 0

    elif a == 1: #if up
      if s[0] == 0: #if the agent is on the top border
        r = -1 #the agent bump the border and s doesn't change
      else:
        s = (s[0]-1, s[1])
        r = 0

    elif a == 2: #if right
      if s[1] == cols-1: #if the agent is on the right border
        r = -1 #the agent bump the border and s doesn't change
      else:
        s = (s[0], s[1]+1)
        r = 0

    elif a == 3: #if down
      if s[0] == rows-1: #if the agent is on the bottom border
        r = -1 #the agent bump the border and s doesn't change
      else:
        s = (s[0]+1, s[1])
        r = 0

    else:
      print("error in generate_actions!")

    if(s == goal):
      r = 1

    A.append(a)
    R.append(r)

  return S, A, R

#Calculates the best action to be selected in state s
def argmax_Q_s_a(s, actions, Q_s_a):
  """
  It calculates the best action which can be taken from s.

  :param s: current state (row,column).
  :param actions: list of available actions.
  :param Q_s_a: dictionary containing the action-value of each pair (s,a).

  :returns: action which maximizes the Q-value in the state s.
  """
  best_a = actions[0]
  best_value = Q_s_a["Q(("+str(s[0])+","+str(s[1])+"),"+str(actions[0])+")"]
  for a in actions[1:]:
    if Q_s_a["Q(("+str(s[0])+","+str(s[1])+"),"+str(actions[0])+")"] > best_value:
      best_a = a
      best_value = Q_s_a["Q(("+str(s[0])+","+str(s[1])+"),"+str(actions[0])+")"]

  return best_a

In [0]:
def on_policy_first_visit_MC_ES(grid_rows = 6, grid_cols = 6, n_episodes = 1000, T = 10, gamma = 0.9, goal = (0,0)):
  """
  It implements the on-policy algorithm of Monte Carlo using the exploration
  starts assumption.

  :param grid_rows: number of rows of the environment. 
  :param grid_cols: number of cols of the environment. 
  :param n_episodes: number of episodes the algorithm will compute.
  :param T: number of steps of each episode.
  :param gamma: discount parameter.
  :param goal: state (row,column) in which the agent receives the reward +1.
  """
  #Simulation parameters
  actions = [0, 1, 2, 3] #left, up, right, down

  #Initialization:
  #1.policy
  policy_grid = np.random.randint(0,3,size=(grid_rows,grid_cols))
  
  #2.Q(s,a)
  #3.Returns(s,a)
  Q_s_a = {}
  returns_s = {}
  for r in range(grid_rows):
    for c in range(grid_cols):
      for a in actions:
        Q_s_a["Q(("+str(r)+","+str(c)+"),"+str(a)+")"] = 0.
        returns_s['returns('+str(r)+','+str(c)+","+str(a)+")"] = []

  print("Initial policy grid:")
  print_policy_grid(policy_grid)

  #Loop for each episode
  for i in range(n_episodes):
    G = 0
    #generate an episode from S_0 amd A_0 (randomly chosen)
    S, A, R = compute_episode_ES(grid_rows,grid_cols,policy_grid,T,goal)

    for t in range(T-1,0,-1): #for each step of the episode
      s = S[t]
      a = A[t]
      G = gamma*G + R[t]
      if s not in S[t-1:-1:-1]: #TODO: check on the pair S_t, A_t  -> removing this check, first-visit becomes every-visit!
        returns_s['returns('+str(s[0])+','+str(s[1])+","+str(a)+")"].append(G)
        Q_s_a["Q(("+str(s[0])+","+str(s[1])+"),"+str(a)+")"] = np.mean(returns_s['returns('+str(s[0])+','+str(s[1])+","+str(a)+")"])
        policy_grid[s[0],s[1]] = argmax_Q_s_a(s, actions, Q_s_a) 
  
  print("\nFinal policy grid:")
  print_policy_grid(policy_grid)


In [23]:
#TEST

#Execution of the on-policy Monte Carlo ES (Exploring Starts)
on_policy_first_visit_MC_ES(grid_rows=5, grid_cols=5,n_episodes = 5000, T = 10)

Initial policy grid:
Policy grid:
[['→' '→' '←' '←' '↑']
 ['→' '→' '→' '←' '←']
 ['↑' '↑' '↑' '→' '→']
 ['↑' '→' '←' '↑' '↑']
 ['←' '→' '←' '←' '←']]

Final policy grid:
Policy grid:
[['←' '←' '←' '←' '←']
 ['←' '←' '←' '←' '←']
 ['←' '←' '←' '←' '←']
 ['←' '←' '←' '←' '←']
 ['←' '→' '←' '←' '←']]


**On-policy first-visit MC control**


![alt text](https://marcinbogdanski.github.io/rl-sketchpad/RL_An_Introduction_2018/assets/0504_OnPolicy_MC_Ctrl.png)

In [0]:
def select_action_epsilon_greedy(s, policy_grid, actions, epsilon):
  """
  It selects an action using epsilon-greedy methods. With probability 1-epsilon,
  it selects the current greedy action, with probability epsilon the action is 
  selected randomly.

  :param s: current state (row,column).
  :param policy_grid: grid showing the current greedy action to be executed.
  :param actions: list of the available actions.
  :param epsilon: parameter defining the exploration.

  :returns: number of the action to execute.
  """
  if(np.random.uniform() > epsilon):
    return policy_grid[s[0],s[1]]
  else:
    return np.random.randint(0,len(actions)-1)

def compute_episode_epsilon_greedy(rows, cols, policy_grid, actions, epsilon, T, goal):
  S = []
  A = []
  R = []

  s = (np.random.randint(0,rows-1),np.random.randint(0,cols-1)) #initialize S_0

  for i in range(T):
    S.append(s)
    a = select_action_epsilon_greedy(s, policy_grid, actions, epsilon)

    if a == 0: #if left
      if s[1] == 0: #if the agent is on the left border
        r = -1 #the agent bump the border and s doesn't change
      else:
        s = (s[0], s[1]-1)
        r = 0

    elif a == 1: #if up
      if s[0] == 0: #if the agent is on the top border
        r = -1 #the agent bump the border and s doesn't change
      else:
        s = (s[0]-1, s[1])
        r = 0

    elif a == 2: #if right
      if s[1] == cols-1: #if the agent is on the right border
        r = -1 #the agent bump the border and s doesn't change
      else:
        s = (s[0], s[1]+1)
        r = 0

    elif a == 3: #if down
      if s[0] == rows-1: #if the agent is on the bottom border
        r = -1 #the agent bump the border and s doesn't change
      else:
        s = (s[0]+1, s[1])
        r = 0

    else:
      print("error in generate_actions!")

    if(s == goal):
      r = 1

    A.append(a)
    R.append(r)

  return S, A, R

In [0]:
def on_policy_first_visit_MC_control(grid_rows = 6, grid_cols = 6, n_episodes = 1000, T = 10, gamma = 0.9, epsilon = 0.5, goal = (0,0)):
  """
  It implements the on-policy algorithm of Monte Carlo using for epsilon-soft
  policies. 

  :param grid_rows: number of rows of the environment. 
  :param grid_cols: number of cols of the environment. 
  :param n_episodes: number of episodes the algorithm will compute.
  :param T: number of steps of each episode.
  :param gamma: discount parameter.
  :param epsilon: exploration paramter.
  :param goal: state (row,column) in which the agent receives the reward +1.
  """
  actions = [0, 1, 2, 3] #left, up, right, down
  
  #1.Initialize arbitrary epsilon-soft policy
  policy_grid = np.random.randint(0,3,size=(grid_rows,grid_cols))

  #2.Q(s,a)
  #3.Returns(s,a)
  Q_s_a = {}
  returns_s = {}
  for r in range(grid_rows):
    for c in range(grid_cols):
      for a in actions:
        Q_s_a["Q(("+str(r)+","+str(c)+"),"+str(a)+")"] = 0.
        returns_s['returns('+str(r)+','+str(c)+","+str(a)+")"] = []

  print("Initial policy grid:")
  print_policy_grid(policy_grid)

  #Loop for each episode
  for i in range(n_episodes):
    G = 0
    #generate an episode using the policy (epsilon-greedy policy)
    S, A, R = compute_episode_epsilon_greedy(grid_rows,grid_cols,policy_grid,actions,epsilon,T,goal)

    for t in range(T-1,0,-1): #for each step of the episode
      s = S[t]
      a = A[t]
      G = gamma*G + R[t]
      if s not in S[t-1:-1:-1]: #TODO: check on the pair S_t, A_t  -> removing this check, first-visit becomes every-visit!
        returns_s['returns('+str(s[0])+','+str(s[1])+","+str(a)+")"].append(G)
        Q_s_a["Q(("+str(s[0])+","+str(s[1])+"),"+str(a)+")"] = np.mean(returns_s['returns('+str(s[0])+','+str(s[1])+","+str(a)+")"])
        policy_grid[s[0],s[1]] = argmax_Q_s_a(s, actions, Q_s_a) 

  print("\nFinal policy grid:")
  print_policy_grid(policy_grid)

In [26]:
#TEST

#Execute on-policy MC algorithm for epsilon-soft policies
on_policy_first_visit_MC_control(grid_rows=5, grid_cols=5, n_episodes= 5000, T= 15, epsilon=0.5)

Initial policy grid:
Policy grid:
[['←' '↑' '→' '←' '←']
 ['→' '↑' '←' '→' '↑']
 ['↑' '→' '↑' '↑' '←']
 ['→' '↑' '↑' '→' '→']
 ['←' '→' '←' '↑' '←']]

Final policy grid:
Policy grid:
[['←' '←' '←' '←' '←']
 ['←' '←' '←' '←' '←']
 ['←' '←' '←' '←' '←']
 ['←' '←' '←' '←' '←']
 ['←' '→' '←' '↑' '←']]


# **Monte Carlo off-policy methods**

In [0]:
def off_policy_MC_prediction(policy = None, grid_rows = 6, grid_cols = 6, n_episodes = 1000, T = 10, gamma = 0.9, goal = (0,0)):
  pass

In [0]:
#TEST

off_policy_MC_prediction()

In [0]:
def off_policy_first_visit_MC_control(grid_rows = 6, grid_cols = 6, n_episodes = 1000, T = 10, gamma = 0.9, epsilon = 0.5, goal = (0,0)):
  pass

In [0]:
#TEST

off_policy_first_visit_MC_control()